In [78]:
# packages
import pandas as pd
from mod02_build_bot_predictor import train_model

### Define a function to extract predictions from the model

In [81]:
def predict_bot(df, model=None):
    """
    Predict whether each account is a bot (1) or human (0).
    """
    if model is None:
        model = train_model()

    preds = model.predict(df)
    return pd.Series(preds, index=df.index)

### Define a function to evaluate model error

In [84]:
def confusion_matrix_and_metrics(y_true, y_pred):
    """
    Computes confusion matrix and common error rates for binary classification.

    Assumes labels:
      0 = negative class
      1 = positive class

    Returns:
      dict with:
        tn, fp, fn, tp
        misclassification_rate
        false_positive_rate
        false_negative_rate
    """
    tn = fp = fn = tp = 0

    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 0:
            tn += 1
        elif yt == 0 and yp == 1:
            fp += 1
        elif yt == 1 and yp == 0:
            fn += 1
        elif yt == 1 and yp == 1:
            tp += 1
        else:
            raise ValueError("Labels must be 0 or 1")

    total = tn + fp + fn + tp

    misclassification_rate = (fp + fn) / total if total > 0 else 0.0
    false_positive_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    false_negative_rate = fn / (fn + tp) if (fn + tp) > 0 else 0.0

    return {
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "misclassification_rate": misclassification_rate,
        "false_positive_rate": false_positive_rate,
        "false_negative_rate": false_negative_rate,
    }


### Load the data

In [87]:
TRAIN_PATH = "mod02_data/train.csv"
train = pd.read_csv(TRAIN_PATH)

TEST_PATH = "mod02_data/test.csv"
test = pd.read_csv(TEST_PATH)

### Format the data by independent vs. dependent variables

In [90]:
X_train = train.drop(columns=["is_bot"])
y_train = train['is_bot']

X_test = test.drop(columns=["is_bot"])
y_test = test['is_bot']

### Build the model on training data

In [93]:
model = train_model(X_train, y_train)

### Get the model predictions on training and test data

In [96]:
y_pred_train = predict_bot(X_train, model)
y_pred_test = predict_bot(X_test, model)

### Check results on the training set (data used to build the model)

In [99]:
confusion_matrix_and_metrics(y_train, y_pred_train)

{'tp': 187,
 'tn': 2625,
 'fp': 12,
 'fn': 176,
 'misclassification_rate': 0.06266666666666666,
 'false_positive_rate': 0.004550625711035267,
 'false_negative_rate': 0.48484848484848486}

### Check results on the test set (new data not yet seen by the model)

In [102]:
confusion_matrix_and_metrics(y_test, y_pred_test)

{'tp': 33,
 'tn': 845,
 'fp': 29,
 'fn': 93,
 'misclassification_rate': 0.122,
 'false_positive_rate': 0.03318077803203661,
 'false_negative_rate': 0.7380952380952381}

# Discussion Questions

### Based on the misclassification rate of your model, discuss your confidence in the ability to predict a bot. 

Based on the misclassicifation rate the model performs reasonably well with a training error of 6.3% but a high error of 12.2% for the training test error. Based on this result it shows that it misses many bots which limits the confidence in detecting them. Improvements in handling the class imbalance or adjusting the decision threshold could increase its accuracy to predict the bots.

### What are potential ramifications of false positives from the model?

False positives happen when the model flags a legitmate user as a bot. It can frustate the user and damage the platform's reputation. Overall, too many false positives reduce and trust effciency.

### What are potential ramifications of false negatives from the model?

False negatvies happen when a model fails to identify an actual bot and mistakes it for a legitmate user. This can allow bots to continue spreading misinformation or malicius content on the platform. As a result, false negatives can harm user experience and overall security. 